In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from  sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from statsmodels.tsa.seasonal import seasonal_decompose

label_encoder = LabelEncoder()


InputLayer = keras.layers.InputLayer()
LSTM = keras.layers.LSTM
Dropout = keras.layers.Dropout
Dense = keras.layers.Dense

In [2]:
read_csv = pd.read_csv('/Users/victoroliveira/Desktop/vonix-py-statistic/src/vonixstatisc/data/pandas_data/data_fluency_queue_no_auto_dialer.csv')

df_to_dict = pd.DataFrame()

df = read_csv.loc[(read_csv['handling_time'] > 10) & (read_csv['handling_time'] < 200) ]
df= df.dropna(axis=0)
df = df.loc[df['queue_id'] == 'equipez4']

df_to_dict['time'] = df['timestamp']
df_to_dict['ht'] = df['handling_time']

In [3]:
dict = df_to_dict.set_index('time').to_dict()['ht']
dict

{1677185676: 78,
 1677186824: 23,
 1677243851: 48,
 1677244092: 53,
 1677250126: 137,
 1677256914: 15,
 1677256993: 99,
 1677257018: 27,
 1677257027: 75,
 1677257032: 13,
 1677257033: 16,
 1677257038: 26,
 1677257050: 16,
 1677257053: 68,
 1677257057: 71,
 1677257058: 104,
 1677257062: 23,
 1677257065: 87,
 1677257069: 51,
 1677257073: 23,
 1677257091: 115,
 1677257094: 47,
 1677257107: 84,
 1677257114: 26,
 1677257124: 21,
 1677257170: 12,
 1677257176: 61,
 1677257187: 55,
 1677257191: 22,
 1677257201: 37,
 1677257202: 118,
 1677257219: 56,
 1677257260: 61,
 1677257277: 26,
 1677257293: 84,
 1677257294: 22,
 1677257297: 54,
 1677257300: 70,
 1677257303: 27,
 1677257304: 53,
 1677257305: 54,
 1677257321: 199,
 1677257349: 26,
 1677257350: 90,
 1677257352: 70,
 1677257359: 72,
 1677257375: 87,
 1677257376: 64,
 1677257379: 51,
 1677257433: 75,
 1677257464: 62,
 1677257477: 95,
 1677257478: 49,
 1677257518: 40,
 1677257519: 91,
 1677257524: 79,
 1677257534: 132,
 1677257548: 60,
 1677257

In [4]:
from datetime import datetime
from time import gmtime, strftime
import time
def transform_dict(data: dict, period: int) -> dict:
    """method that receives dict from database query
    {start_at(timestamp): talk_secs(seconds), ...} and create
    another dict based on the period (15, 30, 1hr) chosen
    {last_timestamp: [12,24,56,12, 90, 100], last_timestamp - period:[195,235,201,300], ... interval of three minutes for standard model}
    For standard model the data from the last period( 15 min, 30 min, 1h) is used to construct forecast for the next period so the return_dict begins with the last timestamp to the first
    """
    seconds = period * 60
    
    return_dict = {}

    keys_array = list(data.keys())
    beginning = keys_array[0]
    limit= beginning + seconds
    end = keys_array[-1]
    arr = []
    break_v = 1
    indice = 0
    while beginning < end:
        if break_v == beginning:
            break
        break_v = beginning
        for key, value in data.items():
            if key <= limit:
                arr.append(value)
                continue
        
            return_dict[f"{indice}"] = arr
            indice += 1
            arr = []
            arr.append(value)
            beginning = key
            limit = beginning + seconds
            

    return return_dict

In [5]:
dict_transformed = transform_dict(dict,120)

In [34]:
dict_transformed

{'0': [78, 23],
 '1': [48, 53, 137],
 '2': [15,
  99,
  27,
  75,
  13,
  16,
  26,
  16,
  68,
  71,
  104,
  23,
  87,
  51,
  23,
  115,
  47,
  84,
  26,
  21,
  12,
  61,
  55,
  22,
  37,
  118,
  56,
  61,
  26,
  84,
  22,
  54,
  70,
  27,
  53,
  54,
  199,
  26,
  90,
  70,
  72,
  87,
  64,
  51,
  75,
  62,
  95,
  49,
  40,
  91,
  79,
  132,
  60,
  54,
  163,
  109,
  30,
  92,
  18,
  111,
  71,
  51,
  65,
  49,
  19,
  75,
  89,
  52,
  42,
  122,
  17,
  42,
  43,
  40,
  49,
  13,
  86,
  41,
  68,
  42,
  35,
  37,
  34,
  41,
  46,
  99,
  86,
  44,
  55,
  58,
  52,
  18,
  168,
  97,
  44,
  43,
  17,
  11,
  95,
  25,
  52,
  46,
  76,
  36,
  48,
  26,
  18,
  13,
  50,
  55,
  36,
  14,
  16,
  19,
  49,
  11,
  53,
  13,
  20,
  126,
  78,
  65,
  66,
  35,
  64,
  52,
  31,
  46,
  92,
  32,
  50,
  70,
  91,
  37,
  34,
  66,
  12,
  52,
  132,
  44,
  68,
  40,
  11,
  160,
  55,
  51,
  43,
  144,
  191,
  16,
  84,
  29,
  28,
  54,
  64,
  52,
  51,
 

In [6]:
mean_object = {i: np.mean(dict_transformed[i]) for i in dict_transformed.keys()}

In [7]:
mean_object

{'0': 50.5,
 '1': 79.33333333333333,
 '2': 54.48837209302326,
 '3': 56.048843187660665,
 '4': 47.6,
 '5': 51.280155642023345,
 '6': 57.36,
 '7': 54.63247863247863,
 '8': 54.91925465838509,
 '9': 53.065400843881854,
 '10': 51.291079812206576,
 '11': 57.833333333333336,
 '12': 22.0,
 '13': 55.81818181818182,
 '14': 58.23076923076923,
 '15': 64.15789473684211,
 '16': 65.66666666666667,
 '17': 37.9375,
 '18': 52.3125,
 '19': 34.0,
 '20': 68.17647058823529,
 '21': 66.63636363636364,
 '22': 60.7,
 '23': 22.555555555555557,
 '24': 53.26086956521739,
 '25': 45.07216494845361,
 '26': 52.578947368421055,
 '27': 42.0,
 '28': 74.5,
 '29': 66.58823529411765,
 '30': 56.89090909090909,
 '31': 41.375,
 '32': 55.0,
 '33': 64.25,
 '34': 53.083333333333336,
 '35': 46.333333333333336,
 '36': 48.833333333333336,
 '37': 27.153846153846153,
 '38': 65.68181818181819,
 '39': 29.0,
 '40': 78.83333333333333,
 '41': 62.86206896551724,
 '42': 63.75,
 '43': 43.357142857142854,
 '44': 38.8,
 '45': 23.0,
 '46': 25.0,

In [8]:
cv_object = {i: np.std(dict_transformed[i]) for i in dict_transformed.keys()}

In [9]:
cv_object

{'0': 27.5,
 '1': 40.827550610940264,
 '2': 35.88910200283029,
 '3': 38.09047786262563,
 '4': 30.43747689937522,
 '5': 35.47793904288466,
 '6': 38.79972901230847,
 '7': 33.09055131248304,
 '8': 39.453404415099094,
 '9': 33.670059963111036,
 '10': 34.67406609264892,
 '11': 41.16126820203673,
 '12': 0.0,
 '13': 51.41775105901601,
 '14': 44.331190350246885,
 '15': 63.37379763866779,
 '16': 51.25504233808967,
 '17': 34.500849174331925,
 '18': 53.00556427159322,
 '19': 27.248853186877426,
 '20': 54.12538307878851,
 '21': 54.284222933593995,
 '22': 49.87394109151592,
 '23': 11.62479257746717,
 '24': 38.646691057303194,
 '25': 29.11202940603318,
 '26': 44.69936510731702,
 '27': 30.940803695659447,
 '28': 37.9835490706174,
 '29': 56.904521502900344,
 '30': 39.516934334201,
 '31': 32.39188748745587,
 '32': 38.79432948254165,
 '33': 23.573024837725004,
 '34': 52.07919983597631,
 '35': 47.54600111704686,
 '36': 34.1951101117624,
 '37': 25.239913915024943,
 '38': 54.835610827965006,
 '39': 16.5126

In [10]:
call_volume = {i: len(dict_transformed[i]) for i in dict_transformed.keys()}

In [11]:
call_volume

{'0': 2,
 '1': 3,
 '2': 387,
 '3': 389,
 '4': 100,
 '5': 257,
 '6': 350,
 '7': 117,
 '8': 161,
 '9': 474,
 '10': 213,
 '11': 18,
 '12': 1,
 '13': 11,
 '14': 13,
 '15': 19,
 '16': 21,
 '17': 16,
 '18': 16,
 '19': 8,
 '20': 17,
 '21': 22,
 '22': 10,
 '23': 9,
 '24': 115,
 '25': 97,
 '26': 19,
 '27': 9,
 '28': 4,
 '29': 17,
 '30': 55,
 '31': 8,
 '32': 4,
 '33': 4,
 '34': 12,
 '35': 15,
 '36': 12,
 '37': 13,
 '38': 22,
 '39': 3,
 '40': 6,
 '41': 29,
 '42': 12,
 '43': 14,
 '44': 5,
 '45': 1,
 '46': 4,
 '47': 1,
 '48': 5,
 '49': 1,
 '50': 1,
 '51': 2,
 '52': 1,
 '53': 63,
 '54': 58,
 '55': 3,
 '56': 3,
 '57': 5,
 '58': 179,
 '59': 281,
 '60': 213,
 '61': 96,
 '62': 115,
 '63': 160,
 '64': 215,
 '65': 1,
 '66': 43,
 '67': 2,
 '68': 6,
 '69': 21,
 '70': 5,
 '71': 4,
 '72': 6,
 '73': 5,
 '74': 4,
 '75': 4,
 '76': 5,
 '77': 3,
 '78': 352,
 '79': 225,
 '80': 116,
 '81': 79,
 '82': 72,
 '83': 203,
 '84': 104,
 '85': 80,
 '86': 68,
 '87': 123,
 '88': 121,
 '89': 48,
 '90': 107,
 '91': 113,
 '92': 1

In [33]:
df_to_ml = pd.DataFrame()
df_to_ml['call_volume'] = call_volume.values()
df_to_ml['cv'] = cv_object.values()
df_to_ml['mean'] = mean_object.values()
df_to_ml

,call_volume,cv,mean
0,2,27.500000,50.500000
1,3,40.827551,79.333333
2,387,35.889102,54.488372
3,389,38.090478,56.048843
4,100,30.437477,47.600000
...,...,...,...
114,125,39.333661,48.848000
115,200,33.337030,49.320000
116,250,33.648441,50.080000
117,4,5.937171,20.500000


In [13]:
df_to_ml.iloc[:, 0:3].values

array([[  2.        ,  27.5       ,  50.5       ],
       [  3.        ,  40.82755061,  79.33333333],
       [387.        ,  35.889102  ,  54.48837209],
       [389.        ,  38.09047786,  56.04884319],
       [100.        ,  30.4374769 ,  47.6       ],
       [257.        ,  35.47793904,  51.28015564],
       [350.        ,  38.79972901,  57.36      ],
       [117.        ,  33.09055131,  54.63247863],
       [161.        ,  39.45340442,  54.91925466],
       [474.        ,  33.67005996,  53.06540084],
       [213.        ,  34.67406609,  51.29107981],
       [ 18.        ,  41.1612682 ,  57.83333333],
       [  1.        ,   0.        ,  22.        ],
       [ 11.        ,  51.41775106,  55.81818182],
       [ 13.        ,  44.33119035,  58.23076923],
       [ 19.        ,  63.37379764,  64.15789474],
       [ 21.        ,  51.25504234,  65.66666667],
       [ 16.        ,  34.50084917,  37.9375    ],
       [ 16.        ,  53.00556427,  52.3125    ],
       [  8.        ,  27.24885

In [14]:
normalizador = MinMaxScaler(feature_range = (0,1))
base_treinamento_normalizada = normalizador.fit_transform(df_to_ml)

In [15]:
base_treinamento_normalizada.shape

(119, 3)

In [17]:
previsores = []
tempo_real = []


In [18]:
for i in range(1, 119):
    previsores.append(base_treinamento_normalizada[i-1:i, 0:2])
    tempo_real.append(base_treinamento_normalizada[i,2])

In [19]:
previsores

[array([[0.00211416, 0.35953174]]),
 array([[0.00422833, 0.53377455]]),
 array([[0.81606765, 0.46920986]]),
 array([[0.82029598, 0.49799039]]),
 array([[0.20930233, 0.39793596]]),
 array([[0.54122622, 0.46383437]]),
 array([[0.73784355, 0.50726305]]),
 array([[0.24524313, 0.43262194]]),
 array([[0.33826638, 0.51580913]]),
 array([[1.        , 0.44019837]]),
 array([[0.44820296, 0.45332463]]),
 array([[0.0359408 , 0.53813754]]),
 array([[0., 0.]]),
 array([[0.02114165, 0.67222958]]),
 array([[0.02536998, 0.57958072]]),
 array([[0.03805497, 0.82854151]]),
 array([[0.0422833 , 0.67010234]]),
 array([[0.03171247, 0.45106001]]),
 array([[0.03171247, 0.69298846]]),
 array([[0.01479915, 0.35624827]]),
 array([[0.03382664, 0.70762884]]),
 array([[0.04439746, 0.70970549]]),
 array([[0.01902748, 0.65204599]]),
 array([[0.01691332, 0.15198116]]),
 array([[0.2410148 , 0.50526225]]),
 array([[0.20295983, 0.38060722]]),
 array([[0.03805497, 0.5843942 ]]),
 array([[0.01691332, 0.4045164 ]]),
 array([

In [20]:
previsores, tempo_real = np.array(previsores), np.array(tempo_real)
previsores= np.reshape(previsores, (previsores.shape[0], previsores.shape[1],2))

In [21]:
previsores.shape[1:]

(1, 2)

In [29]:
regressor = keras.models.Sequential()
#camadas inicial
regressor.add(LSTM(units = 100, return_sequences = True, input_shape=(1,2)))
regressor.add(Dropout(0.3))     

#camadas
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))    

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))  

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3)) 

#camadas finais
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.3)) 

regressor.add(Dense(units = 1, activation = 'sigmoid'))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['mean_absolute_error'])



In [30]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [31]:
es = EarlyStopping(monitor = 'loss', min_delta = 1e-10, patience = 10 , verbose =1)
rlr = ReduceLROnPlateau(monitor = 'loss', factoe = 0.2, patience = 5, verbose = 1)
mcp = ModelCheckpoint(filepath = 'pesos.h5', monitor = 'loss', save_best_only = True)

In [32]:
regressor.fit(previsores, tempo_real, epochs = 100, batch_size = 3)

Epoch 1/100
40/40 [==============================] - 2s 2ms/step - loss: 0.0327 - mean_absolute_error: 0.1412
Epoch 2/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0259 - mean_absolute_error: 0.1164
Epoch 3/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0233 - mean_absolute_error: 0.1052
Epoch 4/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0239 - mean_absolute_error: 0.1080
Epoch 5/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0250 - mean_absolute_error: 0.1095
Epoch 6/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0238 - mean_absolute_error: 0.1073
Epoch 7/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0242 - mean_absolute_error: 0.1089
Epoch 8/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0239 - mean_absolute_error: 0.1058
Epoch 9/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0247 - mean_absolute_error: 0.1086
Epoch 10/1

40/40 [==============================] - 0s 2ms/step - loss: 0.0232 - mean_absolute_error: 0.1041
Epoch 75/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0230 - mean_absolute_error: 0.1033
Epoch 76/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0229 - mean_absolute_error: 0.1037
Epoch 77/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0239 - mean_absolute_error: 0.1055
Epoch 78/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0241 - mean_absolute_error: 0.1073
Epoch 79/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0228 - mean_absolute_error: 0.1022
Epoch 80/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0231 - mean_absolute_error: 0.1059
Epoch 81/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0228 - mean_absolute_error: 0.1026
Epoch 82/100
40/40 [==============================] - 0s 2ms/step - loss: 0.0237 - mean_absolute_error: 0.1056
Epoch 83/100
4